In [89]:
import pandas as pd

In [90]:
df = pd.read_csv('Suicide_Detection.csv', encoding = "utf-8")

In [91]:
df['class'] = df['class'].replace("suicide", 1)
df['class'] = df['class'].replace('non-suicide', 0)

df

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,1
1,3,Am I weird I don't get affected by compliments...,0
2,4,Finally 2020 is almost over... So I can never ...,0
3,8,i need helpjust help me im crying so hard,1
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",1
...,...,...,...
232069,348103,If you don't like rock then your not going to ...,0
232070,348106,You how you can tell i have so many friends an...,0
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,0
232072,348108,The usual stuff you find hereI'm not posting t...,1


In [92]:
import re
from string import punctuation
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [93]:
def depure_data(data):

    #Removing emojis, symbols, maps, flags etc
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    
    data = data.map(lambda x: emoji_pattern.sub(r'', x))
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = data.map(lambda x: url_pattern.sub(r'', x))

    # Remove Emails
    data = data.map(lambda x: re.sub('\S*@\S*\s?', '', x)) 

    # Remove new line characters
    data = data.map(lambda x: re.sub('\s+', ' ', x)) 

    data = data.map(lambda x: ''.join([c for c in x if c not in punctuation]))

    # Remove distracting single quotes
    data = data.map(lambda x: re.sub("\'", "", x)) 

        
    return data

In [94]:
df['text'] = depure_data(df['text'])

In [95]:
text_in_list = df['text'].values.tolist()

In [96]:
import wordninja


In [97]:

temp = []
for i in range(len(text_in_list)):
    temp.append(wordninja.split(text_in_list[i]))
    


temp

[['Ex',
  'Wife',
  'Threatening',
  'Suicide',
  'Recently',
  'I',
  'left',
  'my',
  'wife',
  'for',
  'good',
  'because',
  'she',
  'has',
  'cheated',
  'on',
  'me',
  'twice',
  'and',
  'lied',
  'to',
  'me',
  'so',
  'much',
  'that',
  'I',
  'have',
  'decided',
  'to',
  'refuse',
  'to',
  'go',
  'back',
  'to',
  'her',
  'As',
  'of',
  'a',
  'few',
  'days',
  'ago',
  'she',
  'began',
  'threatening',
  'suicide',
  'I',
  'have',
  'tirelessly',
  'spent',
  'these',
  'pa',
  'at',
  'few',
  'days',
  'talking',
  'her',
  'out',
  'of',
  'it',
  'and',
  'she',
  'keeps',
  'hesitating',
  'because',
  'she',
  'wants',
  'to',
  'believe',
  'Ill',
  'come',
  'back',
  'I',
  'know',
  'a',
  'lot',
  'of',
  'people',
  'will',
  'threaten',
  'this',
  'in',
  'order',
  'to',
  'get',
  'their',
  'way',
  'but',
  'what',
  'happens',
  'if',
  'she',
  'really',
  'does',
  'What',
  'do',
  'I',
  'do',
  'and',
  'how',
  'am',
  'I',
  'supposed

In [98]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [99]:
original_data = []
for i in range(len(temp)):
    original_data.append(detokenize(temp[i]))

In [100]:
original_data

['Ex Wife Threatening Suicide Recently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her As of a few days ago she began threatening suicide I have tirelessly spent these pa at few days talking her out of it and she keeps hesitating because she wants to believe Ill come back I know a lot of people will threaten this in order to get their way but what happens if she really does What do I do and how am I supposed to handle her death on my hands I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure Im worried today may be the day she does it and I hope so much it doesnt happen',
 'Am I weird I dont get affected by compliments if its coming from someone I know irl but I feel really good when internet strangers do it',
 'Finally 2020 is almost over So I can never hear 2020 has been a bad year ever again I swear to fucking God its so annoying',
 'i need help just hel

In [101]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

In [102]:
max_words = 5000
max_len = 200

In [103]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(original_data)
sequences = tokenizer.texts_to_sequences(original_data)
depressions = pad_sequences(sequences, maxlen=max_len)
depressions

array([[   0,    0,    0, ...,    7,  227,  413],
       [   0,    0,    0, ..., 1550,   28,    7],
       [   0,    0,    0, ...,   61,   19, 1024],
       ...,
       [   0,    0,    0, ...,  492,  159,   18],
       [  16,  205, 1511, ...,  101,    4,   84],
       [   0,    0,    0, ...,   18,  543, 1010]])

In [104]:
from keras.layers import Embedding
embedding_layer = Embedding(1000, 64)

LSTM Model

In [105]:
import numpy as np

In [106]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [136]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(2,activation='sigmoid'))

In [137]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [138]:
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

In [139]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [140]:
labels = tf.keras.utils.to_categorical(df['class'], 2, dtype="float32")
labels

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [141]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(depressions, labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

174055 58019 174055 58019


In [142]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [143]:
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
5440/5440 [==============================] - 333s 61ms/step - loss: 0.2211 - accuracy: 0.9187 - val_loss: 0.1798 - val_accuracy: 0.9362

Epoch 00001: val_accuracy improved from -inf to 0.93616, saving model to best_model1.hdf5
Epoch 2/70
5440/5440 [==============================] - 336s 62ms/step - loss: 0.1849 - accuracy: 0.9328 - val_loss: 0.1777 - val_accuracy: 0.9363

Epoch 00002: val_accuracy improved from 0.93616 to 0.93631, saving model to best_model1.hdf5
Epoch 3/70
5440/5440 [==============================] - 340s 62ms/step - loss: 0.1806 - accuracy: 0.9343 - val_loss: 0.1800 - val_accuracy: 0.9365

Epoch 00003: val_accuracy improved from 0.93631 to 0.93649, saving model to best_model1.hdf5
Epoch 4/70
5440/5440 [==============================] - 341s 63ms/step - loss: 0.1784 - accuracy: 0.9346 - val_loss: 0.1746 - val_accuracy: 0.9367

Epoch 00004: val_accuracy improved from 0.93649 to 0.93669, saving model to best_model1.hdf5
Epoch 5/70
5440/5440 [=================